In [1]:
import pandas as pd
import json
from collections import OrderedDict
from itertools import groupby
from datetime import datetime
from pymongo import MongoClient

In [2]:
# Connect
client = MongoClient("mongodb://mongo:27017")

# Connect to `inspector_restaurant` database
db = client['inspector_restaurant']

In [3]:
# NOTE - change the `nrows=None` argument to a smaller number for tesitng
# i.e. `nrows=100` will only load the first 100 rows
# df = pd.read_csv('Food_Service_Establishment_Inspections__Beginning_2005__ACTIVE_.csv', nrows=1000)
df = pd.read_csv('Food_Service_Establishment_Inspections__Beginning_2005__ACTIVE_.csv')

In [4]:
# df.head()

In [5]:
def sanitizeCity(city):
    city = city.strip().title()
    
    # Handles edge cases
    city = city.replace(', Ny', '')
    city = city.replace(' Ny', '')
    city = city.replace(', New York', '')
    city = city.replace('New York', '')
    city = city.replace(', ', '')
    city = city.replace(',', '')
    city = city.replace("'S", "'s")
    
    # Handle typos
    city = city.replace('Sartoga', 'Saratoga')
    city = city.replace('Spirngs', 'Springs')
    city = city.replace('Srpings', 'Springs')
    city = city.replace('Saranc', 'Saranac')
    city = city.replace('Renssealaer', 'Rensselaer')
    city = city.replace('Rensselear', 'Rensselaer')
    city = city.replace('Pomoma', 'Pomona')
    city = city.replace('0Neida', 'Oneida')
    city = city.replace('Wapppingers Falls', 'Wappingers Falls')
    city = city.replace('Wappinger Falls', 'Wappingers Falls')
    city = city.replace('Poughkepsie', 'Poughkeepsie')
    city = city.replace('Pouoghkeepsie', 'Poughkeepsie')
    city = city.replace('Niag Ara', 'Niagara')
    city = city.replace('Niagara Fallls', 'Niagara Falls')
    city = city.replace('Lewistion', 'Lewiston')
    city = city.replace('Cortlant', 'Cortlandt')
    city = city.replace('Cortland Manor', 'Cortlandt Manor')
    city = city.replace('Wellesley Isl.', 'Wellesley Island')
    city = city.replace('Rotterdam Jct', 'Rotterdam Jct.')
    city = city.replace('Blue Mt. Lake', 'Blue Mountain Lake')
    city = city.replace('Blue Mtn Lake', 'Blue Mountain Lake')
    city = city.replace('Bemus Pt.', 'Bemus Point')
    city = city.replace('Annadale On Hudson', 'Annandale-on-Hudson')
    city = city.replace('Annandale-On-Hudson', 'Annandale-on-Hudson')
    city = city.replace('Albany County', 'Albany')
    city = city.replace('Alb Any', 'Albany')
    
    # Returns city
    return city

In [6]:
results = dict()
violationLookup = dict()
errorCount = 0

print('Processing CSV...')

# Iterates over each row in the dataframe
for row in df.iterrows():
    index, data = row
    
    if isinstance(data['ADDRESS'], float):
        # print(data)
        errorCount += 1
        continue
    elif ',  ' not in data['ADDRESS']:
        # print(data)
        errorCount += 1
        continue
    
    # Defines defaultRestaurant object
    defaultRestaurant = dict({
        'facility_id': int(data['NYS HEALTH OPERATION ID']),
        'facility': data['FACILITY'].strip().title().replace("'S","'s"),
        'operation_name': data['OPERATION NAME'].strip().title().replace("'S","'s"),
        'type': data['FOOD SERVICE TYPE'],
        'description': data['FOOD SERVICE DESCRIPTION'],
        'address': dict({
            'street': data['ADDRESS'].split(',  ')[0].strip().title(),
            'city': sanitizeCity(data['ADDRESS'].split(',  ')[1]),
            'state': data['FS FACILITY STATE'].strip().upper(),
            'zipcode': str(data['FACILITY POSTAL ZIPCODE']).strip()
        }),
        'inspections': []
    })
    
    # Gets the parent restaurant, or returns defaultRestaurant
    restaurant = results.get(data['FACILITY'].strip(), defaultRestaurant)

    # Defines the default parent inspection for this violation 
    inspection = dict({
        #'date': datetime.strptime(date, "%m/%d/%y"),
        'date': data['DATE OF INSPECTION'],
        'type': data['INSPECTION TYPE'],
        'comment': data['INSPECTION COMMENTS'] or '',
        'violations': []
    })
    
    # Gets the parent inspection, or returns defaultInspection
    # inspection = restaurant['inspections'].get(data['DATE OF INSPECTION'], defaultInspection)
    hasParent = False
    for insp in restaurant['inspections']:
        if (insp['date'] == inspection['date']):
            inspection = insp
            hasParent = True

    # Adds the individual violation to the inspection record
    inspection['violations'].append(data['VIOLATION ITEM'])
    
    # Tracks the individual violation
    violationLookup[data['VIOLATION ITEM']] = dict({
        'desc': data['VIOLATION DESCRIPTION'],
        'critical': data['CRITICAL VIOLATION'] == 'Not Critical Violation'
    })
        
    # Adds the inspection to the parent restaurant
    # restaurant['inspections'][data['DATE OF INSPECTION']] = inspection
    if (not hasParent):
        restaurant['inspections'].append(inspection)
    
    # Adds the restaurant to our complete list
    results[data['FACILITY'].strip()] = restaurant
print('Done processing.')

Processing CSV...
Done processing.


In [7]:
errorCount

203

In [8]:
# Iterates over each Restaurant and inserts it into Mongo
for k, v in results.items():
    db['restaurants'].insert_one(v)
#     print(json.dumps(v, indent=2)) 

# Logs..
print('Wrote Restaurants to MongoDb...')

Wrote Restaurants to MongoDb...


In [9]:
# Writes the Restaurants to JSON
# file = open('restaurants.json','w') 
# file.write(json.dumps(output, indent=2)) 
# file.close()
# print(json.dumps(violationLookup, indent=2))

In [10]:
# Iterates over each Violation and inserts it into Mongo
for v in violationLookup.items():
    db['violations'].insert_one(dict({ 'vid': v[0], 'desc': v[1]['desc'], 'critical': v[1]['critical'] == 'Critical'}))
    # print(json.dumps(dict({ 'vid': v[0], 'desc': v[1]['desc'], 'critical': v[1]['critical'] == 'Critical'}), indent=2))

# Logs..
print('Wrote Violations to MongoDb...')

Wrote Violations to MongoDb...
